# Déployez un modèle dans le cloud

In [ ]:
# Le projet va être réalisé en 2 temps, dans deux environnements différents.

# 1) Nous allons dans un premier temps développer et exécuter notre code en local, en travaillant sur un nombre limité d'images à traiter.
# 2) Une fois les choix techniques validés, nous déploierons notre solution dans un environnement Big Data en mode distribué.

## 1. Déploiement de la solution en local

In [ ]:
#### execution du code sur google colab ####

In [ ]:
# Installation des packages nécessaires
!pip install Pandas pillow tensorflow pyspark pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=243e6b6772b3ab073d43289a2598fbbdedb42d58abdd71c04692d80938d9e69e
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
# Import des librairies
import pandas as pd
import numpy as np

from PIL import Image

import io
import os

import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split
from pyspark.sql import SparkSession

import multiprocessing as mp
import jax

# Librairires pour credentials et données kaggle
from google.colab import userdata
import os

In [ ]:
# Import du jeu de données Kaggle
# cf -> https://www.kaggle.com/discussions/general/74235

# Credentials Kaggle
os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')
os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')

In [ ]:
# Télécharger les données en zip
! kaggle datasets download -d moltean/fruits

Dataset URL: https://www.kaggle.com/datasets/moltean/fruits
License(s): CC-BY-SA-4.0
 99% 1.27G/1.28G [00:18<00:00, 121MB/s] 
100% 1.28G/1.28G [00:18<00:00, 76.3MB/s]


In [ ]:
# Unzip les données
! unzip 'fruits.zip'

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: fruits-360_dataset/fruits-360/Training/Tomato 3/112_100.jpg  
  inflating: fruits-360_dataset/fruits-360/Training/Tomato 3/113_100.jpg  
  inflating: fruits-360_dataset/fruits-360/Training/Tomato 3/114_100.jpg  
  inflating: fruits-360_dataset/fruits-360/Training/Tomato 3/115_100.jpg  
  inflating: fruits-360_dataset/fruits-360/Training/Tomato 3/116_100.jpg  
  inflating: fruits-360_dataset/fruits-360/Training/Tomato 3/117_100.jpg  
  inflating: fruits-360_dataset/fruits-360/Training/Tomato 3/119_100.jpg  
  inflating: fruits-360_dataset/fruits-360/Training/Tomato 3/120_100.jpg  
  inflating: fruits-360_dataset/fruits-360/Training/Tomato 3/121_100.jpg  
  inflating: fruits-360_dataset/fruits-360/Training/Tomato 3/122_100.jpg  
  inflating: fruits-360_dataset/fruits-360/Training/Tomato 3/123_100.jpg  
  inflating: fruits-360_dataset/fruits-360/Training/Tomato 3/124_100.jpg  
  inflating: fruits-36

In [ ]:
# On ne prend q'un extrait de 300 images à traiter dans cette première version en local
# L'extrait des images à charger est stockée dans le dossier "Test1"

# pas de dossier "Test1" mais dossier "Test" -> prendre celui là ???

# Nous enregistrons le résultat du traitement dans le dossier 'Results_local'

In [ ]:
PATH = os.getcwd()
PATH_Data = PATH + '/fruits-360-original-size/fruits-360-original-size/Test'
PATH_Result = PATH + '/fruits-360-original-size/fruits-360-original-size/Results_Local'
print('PATH:        ' + PATH + '\nPATH_Data:   ' + PATH_Data + '\nPATH_Result: ' + PATH_Result)

PATH:        /content
PATH_Data:   /content/fruits-360-original-size/fruits-360-original-size/Test
PATH_Result: /content/fruits-360-original-size/fruits-360-original-size/Results_Local


In [ ]:
########## Création de la session Spark (SparkSession) ##########

# Une SparkSession
# = processus de pilotage (driver process) qui contrôle l'application Spark
# = façon dont Spark exécute les fonctions définies par l'utilisateur dans l'ensemble du cluster

In [ ]:
# définir la méthode de démarrage du multiprocessing sur spawn
mp.set_start_method('spawn', force = True)

In [ ]:
# 'os.fork()' -> incompatible avec JAX (bibliothèque pour le calcul numérique basée sur des tableaux)
# désactiver la compilation JIT (Just-In-Time)
jax.config.update('jax_disable_jit', True)

In [ ]:
spark = (
    SparkSession
    .builder
    .appName('P8') # nom de l'application qui sera affiché dans l'interface utilisateur Web Spark
    .master('local') # pour que l'application s'execute localement
    .config('spark.sql.parquet.writeLegacyFormat', 'true') # choix du format "parquet" pour enregistrer le résultat de notre travail
    .getOrCreate() # obtenir une session spark existante ou en créer une nouvelle si n'existe pas
)

# Nous ne définissons pas le nombre de cœurs à utiliser (comme .master('local[4]) pour 4 cœurs à utiliser), nous utiliserons donc tous les cœurs disponibles dans notre processeur

/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


In [ ]:
# Nous créons également la variable "sc" qui est un SparkContext issue de la variable spark :
sc = spark.sparkContext

In [ ]:
# Affichage des informations de Spark en cours d'execution :
spark

In [ ]:
# Les images sont chargées au format binaire, ce qui offre, plus de souplesse dans la façon de prétraiter les images.
# Avant de charger les images, nous spécifions que nous voulons charger uniquement les fichiers dont l'extension est jpg.
# Nous indiquons également de charger tous les objets possibles contenus dans les sous-dossiers du dossier communiqué.

images = spark.read.format('binaryFile') \
  .option('pathGlobFilter', '*.jpg') \
  .option('recursiveFileLookup', 'true') \
  .load(PATH_Data)

In [ ]:
# Affichage des 5 premières images contenant:
# - le path de l'image
# - la date et heure de sa dernière modification
# - sa longueur
# - son contenu encodé en valeur hexadécimal

In [ ]:
# Je ne conserve que le path de l'image et j'ajoute une colonne contenant les labels de chaque image :
images = images.withColumn('label', element_at(split(images['path'], '/'), -2))
print(images.printSchema())
print(images.select('path','label').show(5, False))

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+-------------------------------------------------------------------------------------------+-----------+
|path                                                                                       |label      |
+-------------------------------------------------------------------------------------------+-----------+
|file:/content/fruits-360-original-size/fruits-360-original-size/Test/apple_hit_1/r0_115.jpg|apple_hit_1|
|file:/content/fruits-360-original-size/fruits-360-original-size/Test/apple_hit_1/r0_119.jpg|apple_hit_1|
|file:/content/fruits-360-original-size/fruits-360-original-size/Test/apple_hit_1/r0_107.jpg|apple_hit_1|
|file:/content/fruits-360-original-size/fruits-360-original-size/Test/apple_hit_1/r0_143.jpg|apple_hit_1|
|file:/content/fruits-360-original-size/fruits-3

In [ ]:
########## Préparation du modèle ##########

In [ ]:
# 1) On charge le modèle MobileNetV2 avec les poids précalculés issus d'imagenet et on spécifie le format de nos images en entrée
model = MobileNetV2(
    weights = 'imagenet',
    include_top = True,
    input_shape = (224, 224, 3)
    )

14536120/14536120 [==============================] - 0s 0us/step


In [ ]:
# 2) On créé un nouveau modèle avec:
# - en entrée : l'entrée du modèle MobileNetV2
# - en sortie : l'avant dernière couche du modèle MobileNetV2

new_model = Model(
    inputs = model.input,
    outputs = model.layers[-2].output
    )

In [ ]:
# Affichage du résumé de notre nouveau modèle où nous constatons que nous récupérons bien en sortie un vecteur de dimension (1, 1, 1280)
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']        

In [ ]:
# Tous les workeurs doivent pouvoir accéder au modèle ainsi qu'à ses poids.
# Une bonne pratique consiste à charger le modèle sur le driver puis à diffuser ensuite les poids aux différents workeurs.

brodcast_weights = sc.broadcast(new_model.get_weights())

In [ ]:
# Même chose mais sous forme de fonction :

def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(
        weights = 'imagenet',
        include_top = True,
        input_shape = (224, 224, 3)
        )
    for layer in model.layers:
        layer.trainable = False

    new_model = Model(inputs = model.input,
                  outputs = model.layers[-2].output)

    new_model.set_weights(brodcast_weights.value)

    return new_model

In [ ]:
########## Définition du processus de chargement des images et application de leur featurisation à travers l'utilisation de pandas UDF ##########

# pandas UDF
# featuriser une série d'images pd.Series
# prétraiter une images

In [ ]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [ ]:
########## Exécution des actions d'extration de features #########


In [ ]:
# Commande à exécuter si erreur de type Out Of Memory (OOM) - (par exemple avec de très grandes images)

# spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

In [ ]:
### Exécuter la featurisation sur l'ensemble de notre dataframe spark ###

# peut être chronophage quand gros volume de données à traiter
# ici, jeu de données Test = 22819 images
# on prend un ensemble réduit de 330 images

features_df = images.repartition(20).select(
    col('path'),
    col('label'),
    featurize_udf('content').alias('features')
    )

In [ ]:
features_df.head()

In [ ]:
# Rappel du PATH où seront inscrits les fichiers au format parquet contenant nos résultats, à savoir un df contenant 3 colonnes
# colonne 1 = path des images
# colonne 2 = label de l'image
# colonne 3= vecteur de caractéristiques de l'image

print(PATH_Result)

/content/fruits-360-original-size/fruits-360-original-size/Results_Local


In [ ]:
######## ACP ################

# FAIRE ACP ICI ? ou après ?

# acp sur pyspark pas comme sur sklearn
# soit double acp pour avoir bien avoir la bonne valeur (d'abord acp -> valeurs propres garder 90% ouis nouvelle acp)

In [ ]:
# Enregistrement des données traitées au format "parquet" :
features_df.write.mode('overwrite').parquet(PATH_Result)

In [ ]:
# Conversion des caractéristiques en vecteurs
to_vector_udf = udf(lambda features: Vectors.dense(features), VectorUDT())
features_vector_df = features_df.withColumn('features_vector', to_vector_udf('features'))

In [ ]:
# Détermination du k optimal
variance_df = []
for k in range(1, 201, 50):
    print(f"Calculating PCA for k = {k}")
    pca = PCA(k = k, inputCol = 'features_vector', outputCol = 'pca_features')
    model = pca.fit(features_vector_df)
    explained_variance = model.explainedVariance.sum()
    variance_df.append((k, explained_variance))

In [ ]:
# Convertir en DataFrame pour trouver le k optimal
variance_spark_df = spark.createDataFrame(variance_df, ['k', 'explained_variance'])
optimal_k = variance_spark_df.orderBy(variance_spark_df.explained_variance.desc()).first()[0]

print(f'Optimal number of components: {optimal_k}')

In [ ]:
# Appliquer l'ACP avec le k optimal
pca = PCA(k = optimal_k, inputCol = 'features_vector', outputCol = 'pca_features')
pca_model = pca.fit(features_vector_df)
pca_result_df = pca_model.transform(features_vector_df)

In [ ]:
# Sélection des colonnes nécessaires
result_df = pca_result_df.select('path', 'label', 'pca_features')

In [ ]:
# Enregistrement des résultats réduits
result_df.write.mode('overwrite').parquet(PATH_Result + '/pca_results')

In [ ]:
result_df.head()

In [ ]:
result_df.loc[0,'features'].shape

In [ ]:
########## Chargement des données enregistrées et validation du résultat ##########

In [ ]:
# On charge les données fraichement enregistrées dans un DataFrame Pandas
df = pd.read_parquet(PATH_Result, engine = 'pyarrow')

In [ ]:
df_pca = pd.read_parquet((PATH_Result + '/pca_results'), engine = 'pyarrow')

In [ ]:
# On affiche les 5 premières lignes du DataFrame :
df.head()

,path,label,features
0,file:/content/fruits-360-original-size/fruits-...,apple_hit_1,"[0.77332884, 0.67616427, 2.2709684, 0.0, 0.177..."
1,file:/content/fruits-360-original-size/fruits-...,apple_hit_1,"[0.40961808, 0.36643082, 1.8901002, 0.0, 0.0, ..."
2,file:/content/fruits-360-original-size/fruits-...,apple_hit_1,"[0.117370725, 0.44504938, 1.1451559, 0.0, 0.05..."
3,file:/content/fruits-360-original-size/fruits-...,apple_hit_1,"[0.2088815, 0.16807964, 0.3648796, 0.0, 0.0077..."
4,file:/content/fruits-360-original-size/fruits-...,apple_hit_1,"[0.007770694, 0.1888832, 0.034584805, 0.0, 0.0..."


In [ ]:
# On valide que la dimension du vecteur de caractéristiques des images est bien de dimension 1280 :
df.loc[0,'features'].shape

(1280,)

In [ ]:
----------- fin de l'exécution du code en local -----------

SyntaxError: unterminated string literal (detected at line 1) (<ipython-input-37-337e47fafb04>, line 1)

In [ ]:
# Nous venons de valider le processus sur un jeu de données allégé en local où nous avons simulé un cluster de machines en répartissant la charge de travail
# sur différents cœurs de processeur au sein d'une même machine.

# Nous allons maintenant généraliser le processus en déployant notre solution sur un réel cluster de machines et nous travaillerons désormais sur la totalité
# des 22819 images de notre dossier "Test".

# 2. Déploiement de la solution sur le cloud

In [ ]:
########## Démarrage de la session Spark ##########

In [ ]:
# L'exécution de cette cellule démarre l'application Spark

In [ ]:
# Affichage des informations sur la session en cours et liens vers Spark UI:
%%info

In [ ]:
# Import des packages
# Les packages nécessaires ont été installé via l'étape de bootstrap à l'instanciation du serveur (bootstrap-emr.sh)

# Import des librairies
import pandas as pd
from PIL import Image
import numpy as np
import io
import os

import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.layers import Dense

from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from keras.models import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split
from pyspark.sql import SparkSession
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
from pyspark import SparkConf, SparkContext

In [ ]:
# Initialisation des chemins
PATH = 's3://p8-lucile-data/'
PATH_Data = PATH + '/Test'
PATH_Result = PATH + '/Results'

In [ ]:
########## Traitement des données ##########

In [ ]:
# Chargement des données
images = spark.read.format('binaryFile') \
  .option('pathGlobFilter', '*.jpg') \
  .option('recursiveFileLookup', 'true') \
  .load(PATH_Data)

In [ ]:
images.show(5)

In [ ]:
# On ne conserve que le path de l'image et on ajoute une colonne contenant les labels de chaque image

In [ ]:
images = images.withColumn('label', element_at(split(images['path'], '/'), -2))
print(images.printSchema())
print(images.select('path', 'label').show(10, False))

In [ ]:
# Préparation du modèle
model = MobileNetV2(
    weights = 'imagenet',
    include_top = True,
    input_shape = (224, 224, 3)
    )

In [ ]:
new_model = Model(
    inputs = model.input,
    outputs = model.layers[-2].output
    )

In [ ]:
brodcast_weights = sc.broadcast(new_model.get_weights())

In [ ]:
new_model.summary()

In [ ]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed
    and broadcasted pretrained weights.
    """

    model = MobileNetV2(
        weights = 'imagenet',
        include_top = True,
        input_shape = (224, 224, 3)
        )

    for layer in model.layers:
        layer.trainable = False

    new_model = Model(
        inputs = model.input,
        outputs = model.layers[-2].output
        )
    new_model.set_weights(brodcast_weights.value)

    return new_model

In [ ]:
# Définition du processus de chargement des images et application de leur featurisation à travers l'utilisation de pandas UDF

In [ ]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

In [ ]:
########## Exécution des actions d'extractions de features ##########

In [ ]:
# spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

In [ ]:
features_df = images.repartition(24).select(
    col('path'),
    col('label'),
    featurize_udf('content').alias('features')
    )

In [ ]:
features_df.write.mode("overwrite").parquet(PATH_Result)

In [ ]:
# Conversion des caractéristiques en vecteurs
to_vector_udf = udf(lambda features: Vectors.dense(features), VectorUDT())
features_vector_df = features_df.withColumn('features_vector', to_vector_udf('features'))

In [ ]:
# Détermination du k optimal
variance_df = []
for k in range(1, 201, 50):
    print(f"Calculating PCA for k = {k}")
    pca = PCA(k = k, inputCol = 'features_vector', outputCol = 'pca_features')
    model = pca.fit(features_vector_df)
    explained_variance = model.explainedVariance.sum()
    variance_df.append((k, explained_variance))

In [ ]:
# Convertir en DataFrame pour trouver le k optimal
variance_spark_df = spark.createDataFrame(variance_df, ['k', 'explained_variance'])
optimal_k = variance_spark_df.orderBy(variance_spark_df.explained_variance.desc()).first()[0]

print(f'Optimal number of components: {optimal_k}')

In [ ]:
# Appliquer l'ACP avec le k optimal
pca = PCA(k = optimal_k, inputCol = 'features_vector', outputCol = 'pca_features')
pca_model = pca.fit(features_vector_df)
pca_result_df = pca_model.transform(features_vector_df)

In [ ]:
# Sélection des colonnes nécessaires
result_df = pca_result_df.select('path', 'label', 'pca_features')

In [ ]:
# Enregistrement des résultats réduits
result_df.write.mode('overwrite').parquet(PATH_Result + '/pca_results')

In [ ]:
result_df.head()

In [ ]:
result_df.shape

In [ ]:
result_df.loc[0,'features'].shape

In [ ]:
########## Chargement des données enregistrées et validation du résultat ##########

In [ ]:
df = spark.read.parquet(PATH_Result)
#df = pd.read_parquet(PATH_Result, engine = 'pyarrow')

In [ ]:
df_pca = spark.read.parquet(PATH_Result + '/pca_results')

In [ ]:
df.head()

In [ ]:
print(df.printSchema())

In [ ]:
print(df_pca.printSchema())

In [ ]:
# Sélectionner la colonne "pca_features" et la renommer en "features"
df_pca_features = df_pca.select(col('pca_features').alias('features'))

# Récupérer la première ligne du DataFrame
first_row = df_pca_features.first()

# Obtenir la dimension de la colonne "features"
dimension = len(first_row['features'])
print('Dimension après réduction PCA:', dimension)

In [ ]:
df_pca.loc[0,'features'].shape

In [ ]:
df.loc[0,'features'].shape

In [ ]:
df.columns

In [ ]:
df_pca.columns

In [ ]:
df.count()

In [ ]:
df_pca.count()

In [ ]:
df.shape